# Libraries

In [156]:
import torch
import torchvision
import torch.nn as nn
from torchinfo import summary
from torchvision.io import read_image
from torchvision.transforms import transforms
import numpy as np

# Default Device

In [4]:
if torch.cuda.is_available(): print(torch.cuda.get_device_name()) 
else: print('cpu')
default_device = 'cuda' if torch.cuda.is_available() else 'cpu'

NVIDIA GeForce GTX 1660 Ti


# Model - Feature Extractor

In [32]:
VGG = torchvision.models.vgg16(weights=torchvision.models.VGG16_Weights.IMAGENET1K_V1)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\Alireza/.cache\torch\hub\checkpoints\vgg16-397923af.pth
100.0%


In [157]:
class FeatureExtractor(nn.Module):

    def __init__(self):
        super(FeatureExtractor, self).__init__()
        self.feature = VGG.features
        self.classifier = VGG.classifier[:5] 

    def forward(self, x):
        x = self.feature(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x
    
    def extract(self, x):
        x = self.forward(x)
        x = x.cpu().detach().numpy()
        x = (x / (np.linalg.norm(x,axis=1))[:, np.newaxis])
        return x

In [158]:
feature_extractor = FeatureExtractor().to(default_device)

In [159]:
summary(feature_extractor, input_size=(1, 3, 224, 224))

Layer (type:depth-idx)                   Output Shape              Param #
FeatureExtractor                         [1, 4096]                 --
├─Sequential: 1-1                        [1, 512, 7, 7]            --
│    └─Conv2d: 2-1                       [1, 64, 224, 224]         1,792
│    └─ReLU: 2-2                         [1, 64, 224, 224]         --
│    └─Conv2d: 2-3                       [1, 64, 224, 224]         36,928
│    └─ReLU: 2-4                         [1, 64, 224, 224]         --
│    └─MaxPool2d: 2-5                    [1, 64, 112, 112]         --
│    └─Conv2d: 2-6                       [1, 128, 112, 112]        73,856
│    └─ReLU: 2-7                         [1, 128, 112, 112]        --
│    └─Conv2d: 2-8                       [1, 128, 112, 112]        147,584
│    └─ReLU: 2-9                         [1, 128, 112, 112]        --
│    └─MaxPool2d: 2-10                   [1, 128, 56, 56]          --
│    └─Conv2d: 2-11                      [1, 256, 56, 56]          29

# Data Processing

In [74]:
transform = transforms.Compose([transforms.Resize((224,224)),
                                transforms.ConvertImageDtype(torch.float),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [208]:
lion1 = transform(read_image("C:/Users/Alireza/Desktop/1.jpg"))
lion2 = transform(read_image("C:/Users/Alireza/Desktop/2.jpg"))
fox = transform(read_image("C:/Users/Alireza/Desktop/3.jpeg"))

q_batch = torch.reshape(lion1,[1,3,224,224])
d_batch = torch.stack((lion2,fox))
f_q = feature_extractor.extract(q_batch.to(default_device))
f_d = feature_extractor.extract(d_batch.to(default_device))

# Distance

In [209]:
def distance(f_q, f_d):
    dist = []
    for feature in f_d:
        dist.append(np.linalg.norm(f_q-feature))
        indx = np.argmin(np.array(dist))
    return indx, dist[indx]

In [210]:
distance(f_q, f_d)

(0, 1.0426457)